In [1]:
import pandas as pd
import json
from sklearn.cluster import KMeans

# Cluster users
user_info = json.load(open("aus_user_feature.json"))
user_info = dict(user_info)
# extract the data, extract the value of the dictionary
user_info_value = [user_info[k] for k in user_info.keys()]
kmeans = KMeans(n_clusters=16,  random_state=42, max_iter = 300).fit(user_info_value)

In [2]:
user_label = {list(user_info.keys())[i]:kmeans.labels_[i] for i in range(len(kmeans.labels_))}
user_label_df = pd.DataFrame(user_label.items(), columns=["user", "label"])
user_df = pd.DataFrame(user_info.items(),columns=["user","weights"])
user_df_with_label = pd.merge(user_df, user_label_df, how = "left", on=["user"])

In [3]:
aus_user_item = pd.read_csv("aus_user_item.csv",index_col= [0])
aus_user_item = aus_user_item.reset_index()
aus_user_item = aus_user_item[["steam_id","item_id"]]
aus_user_item["steam_id"] = aus_user_item["steam_id"].astype(str)
aus_user_item["item_id"] = aus_user_item["item_id"].astype(str)
aus_user_item = aus_user_item.drop_duplicates()


In [4]:
raw_bundle_item = pd.read_json("../data/bundle_data.json")
raw_bundle_item = raw_bundle_item.explode("items")
raw_bundle_item = raw_bundle_item.reset_index()
raw_bundle_item["item_id"] = raw_bundle_item.apply(lambda x: x["items"]["item_id"],axis=1)
raw_bundle_item = raw_bundle_item[["bundle_id","item_id"]]
complex_item_data = raw_bundle_item[raw_bundle_item["item_id"].str.contains(",")].copy()
print(len(complex_item_data))
new_item_data = []
for index,row in complex_item_data.iterrows():
    item_id_list = row["item_id"].split(",")
    for item_id in item_id_list:
        new_row = row.copy()
        new_row["item_id"] = item_id
        new_item_data.append(new_row)
new_item_data_df = pd.DataFrame(new_item_data)
item_data_df = raw_bundle_item.drop(raw_bundle_item[raw_bundle_item["item_id"].str.contains(",")].index)
item_data_df = pd.concat([item_data_df, new_item_data_df])
item_data_df = item_data_df.drop_duplicates()

user_bundle = pd.merge(aus_user_item, item_data_df, how='left', on = ["item_id"])
user_bundle = user_bundle.drop_duplicates(["steam_id","item_id"])
user_bundle = user_bundle.dropna()
user_bundle = user_bundle[["steam_id", "bundle_id"]]
user_bundle = user_bundle.drop_duplicates()
user_bundle['steam_id'] = user_bundle['steam_id'].astype(str)
user_bundle["bundle_id"] = user_bundle["bundle_id"].astype(str)
user_bundle["bundle_id"] = user_bundle["bundle_id"].apply(lambda x: x.replace(".0",""))

14


In [5]:
user_label_df = user_label_df.rename({"user":"steam_id"},axis=1)
user_bundle_with_user_label = pd.merge(user_bundle, user_label_df, how = 'left', on = ['steam_id'])

In [6]:
user_bundle_with_user_label = user_bundle_with_user_label.dropna()
user_bundle = user_bundle.dropna()
user_bundle = user_bundle.drop_duplicates()
user_bundle_with_user_label["label"] = user_bundle_with_user_label["label"].astype(int)

In [7]:
breadth_bundle = user_bundle_with_user_label.groupby("bundle_id").size()/len(user_info)

In [8]:
from collections import defaultdict
depth_bundle = defaultdict(int)
for b_id in user_bundle_with_user_label.bundle_id.unique():
    bundle_data = user_bundle_with_user_label[user_bundle_with_user_label['bundle_id'] == b_id]
    purchasing_user = len(bundle_data)
    largest_group = bundle_data.groupby("label").size().max()
    depth_bundle[b_id] = largest_group/purchasing_user


In [9]:
depth_bundle_df= pd.DataFrame(depth_bundle.items(),columns=["bundle_id","depth_coverage"])

In [10]:
breadth_bundle_df = pd.DataFrame(breadth_bundle).reset_index()

In [11]:
breadth_bundle_df.rename({0:"breadth_coverage"}, inplace=True,axis=1)

In [12]:
bundle_features = pd.merge(depth_bundle_df,breadth_bundle_df, on=["bundle_id"], how = "left")

In [13]:
bundle_features.to_csv("features_dataset/bundle_features_with_depth.csv")